In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [5]:
train.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [6]:
test.isna().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [7]:
train.Age.fillna(train.Age.mean(), inplace=True)
test.Age.fillna(test.Age.mean(), inplace=True)

In [8]:
train.Cabin.fillna(train.Cabin.mode()[0], inplace=True)
test.Cabin.fillna(test.Cabin.mode()[0], inplace=True)

In [9]:
train.Embarked.fillna(train.Embarked.mode()[0], inplace=True)

In [10]:
test.Fare.fillna(train.Fare.mean(), inplace=True)

In [11]:
test.isna().sum()

PassengerId    0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

In [12]:
cat_col = []
for col in train.columns:
    if train[col].dtype == "object":
        cat_col.append(col)

#LabelEncoder
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
for column in cat_col:
    train[column] = lb.fit_transform(train[column])
    test[column] = lb.fit_transform(test[column])

In [13]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,108,1,22.0,1,0,523,7.2500,47,2
1,2,1,1,190,0,38.0,1,0,596,71.2833,81,0
2,3,1,3,353,0,26.0,0,0,669,7.9250,47,2
3,4,1,1,272,0,35.0,1,0,49,53.1000,55,2
4,5,0,3,15,1,35.0,0,0,472,8.0500,47,2


In [14]:
X = train.drop(['Survived', 'PassengerId'], axis=1).values
y = train.Survived.values

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=0)

In [16]:
import lightgbm as lgb
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

In [17]:
params = {
        'task': 'train',
        'objective': 'binary',
        'metric': 'binary_error',
        'verbose': 1
}

In [18]:
gbm = lgb.train(params,
            lgb_train,
            num_boost_round=1,
            early_stopping_rounds=100,
            valid_sets=[lgb_train,lgb_eval],
            valid_names=['train', 'eval'])

[1]	train's binary_error: 0.388443	eval's binary_error: 0.373134
Training until validation scores don't improve for 100 rounds.
Did not meet early stopping. Best iteration is:
[1]	train's binary_error: 0.388443	eval's binary_error: 0.373134


In [19]:
prediction = gbm.predict(X_test)

In [20]:
encodedPrediction = [1 if x > 0.50 else 0 for x in prediction]    

In [21]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [22]:
print(accuracy_score(y_test, encodedPrediction))
print(confusion_matrix(y_test, encodedPrediction))

0.6268656716417911
[[168   0]
 [100   0]]


In [23]:
import lgbmTrainingAPI_to_pmml

In [24]:
filename = 'lgbmExportedPMM.pmml'

In [25]:
lgbmTrainingAPI_to_pmml.Export(model = gbm , pmml_f_name = filename)

In [26]:
jsondata = gbm.dump_model()
import pprint
pp = pprint.PrettyPrinter()
pp.pprint(jsondata)

{'average_output': False,
 'feature_names': ['Column_0',
                   'Column_1',
                   'Column_2',
                   'Column_3',
                   'Column_4',
                   'Column_5',
                   'Column_6',
                   'Column_7',
                   'Column_8',
                   'Column_9'],
 'label_index': 0,
 'max_feature_idx': 9,
 'name': 'tree',
 'num_class': 1,
 'num_tree_per_iteration': 1,
 'objective': 'binary sigmoid:1',
 'pandas_categorical': None,
 'tree_info': [{'num_cat': 0,
                'num_leaves': 22,
                'shrinkage': 1,
                'tree_index': 0,
                'tree_structure': {'decision_type': '<=',
                                   'default_left': True,
                                   'internal_count': 623,
                                   'internal_value': 0,
                                   'left_child': {'decision_type': '<=',
                                                  'default_left

In [27]:
gbm.save_model('textModel.txt')

In [28]:
tree_info = jsondata['tree_info']

In [29]:
pp.pprint(tree_info)

[{'num_cat': 0,
  'num_leaves': 22,
  'shrinkage': 1,
  'tree_index': 0,
  'tree_structure': {'decision_type': '<=',
                     'default_left': True,
                     'internal_count': 623,
                     'internal_value': 0,
                     'left_child': {'decision_type': '<=',
                                    'default_left': True,
                                    'internal_count': 215,
                                    'internal_value': 1.53668,
                                    'left_child': {'decision_type': '<=',
                                                   'default_left': True,
                                                   'internal_count': 122,
                                                   'internal_value': 2.29834,
                                                   'left_child': {'decision_type': '<=',
                                                                  'default_left': True,
                                       

In [30]:
newgbm = lgb.basic.Booster(params = {'model_str' : open('textModel.txt', "r").read()})

In [31]:
frcprediction = newgbm.predict(X_test)

In [32]:
frcencodedPrediction = [1 if x > 0.50 else 0 for x in frcprediction] 

In [33]:
print(accuracy_score(y_test, frcencodedPrediction))
print(confusion_matrix(y_test, frcencodedPrediction))

0.6268656716417911
[[168   0]
 [100   0]]


In [34]:
0.6268656716417911

0.6268656716417911

In [35]:
newgbm.save_model('frctextModel.txt')

#PMML RECONSTRUCT

In [36]:
import pmml_to_lgbmTrainAPI

In [37]:
reconstructed_model = pmml_to_lgbmTrainAPI.reconstruct(filename)

184.16000366210938 38.869598388671875 2.3490400314331055 1.6586999893188477 2.826240062713623 2.211280107498169 1.247380018234253 1.0003199577331543 0.6706719994544983 0.6047520041465759 0.5679550170898438 7.105429898699844e-15 16.622299194335938 7.105429898699844e-15 14.97700023651123 13.695300102233887 7.435050010681152 7.217080116271973 5.051459789276123 3.10332989692688 2.8340699672698975


In [38]:
rcprediction = reconstructed_model.predict(X_test)

In [39]:
rcencodedPrediction = [1 if x > 0.50 else 0 for x in rcprediction]

In [40]:
print(accuracy_score(y_test, rcencodedPrediction))
print(confusion_matrix(y_test, rcencodedPrediction))

0.6268656716417911
[[168   0]
 [100   0]]


In [41]:
# model has not been reconstructed properly 
# Reason
# Left Child and Right Child values
# Order of other tree values
# TODO
# Compare Tempfile.txt and textModel.txt